### I Used this Notebook to analyze the files and generate functions to correctly extract the individual attributes from the input Thats also the reason why this notebook is a mess

In [1]:
import pandas as pd
import numpy as np
import scrapy
from scrapy.selector import Selector
from datetime import datetime, timedelta
import locale
import re
locale.setlocale(locale.LC_TIME, 'de_DE')
pd.set_option('display.max_colwidth', None)

In [2]:
import os

def find_attributes(dir_path):
    attributes = set()

    for filename in os.listdir(dir_path):
        if filename.endswith('.html'):
            file_path = os.path.join(dir_path, filename)
            
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            selector = scrapy.Selector(text=content)
            
            # Get all the attribute rows which contains fiche_rando_b
            attribute_rows = selector.xpath('//td[@class="fiche_rando_b"]/text()').getall()
            
            attributes.update(attr.strip() for attr in attribute_rows)
    exclude_attributes = [
        'reine Aufstiegszeit:',
        'Entfernung ungefähr (ebenenprojiziert):',
        'Wetterverhältnisse:',
        'Wegmarkierung:',
        'Gipfelbuch:',
        'Hilfsmittel:',
        'Sonstiges:'
    ]
    attributes = attributes - set(exclude_attributes)
    return attributes


dir_path = './data/raw/200posts/'

unique_attributes = find_attributes(dir_path)

print(unique_attributes)

{'Hochtouren Schwierigkeit:', 'Ski Schwierigkeit:', 'Geo-Tags:', 'Klettern Schwierigkeit:', 'Region:', 'Unterkunftmöglichkeiten:', 'Strecke:', 'Wandern Schwierigkeit:', 'Tour Datum:', 'Zeitbedarf:', 'Abstieg:', 'Kartennummer:', 'Aufstieg:', 'Klettersteig Schwierigkeit:', 'Mountainbike Schwierigkeit:', 'Schneeshuhtouren Schwierigkeit:', 'Wegpunkte:', 'Zufahrt zum Ausgangspunkt:', 'Zufahrt zum Ankunftspunkt:'}


In [3]:
def parse_raw(tour, attributes):
    [id, text] = tour

    #id:  ./data/raw/200posts/post24001.html
    tour_id = id.split('/')[-1].split('.')[0].replace('post', '')
    # Parse it using scrapy
    document = Selector(text=text)

    tour_id = id.split('/')[-1].split('.')[0].replace('post', '')
    # Parse it using scrapy
    document = Selector(text=text)
    # Do some extraction

    # TODO: Extract more attributes and add them to the result dictionary!
    publishing_date_str = document.css('div.author::text').re_first(r'\d{1,2}\. \w+ \d{4} um \d{2}:\d{2}')
    publishing_date = datetime.strptime(publishing_date_str, '%d. %B %Y um %H:%M') if publishing_date_str else None
    author_id = document.css('img[id^="anchor_author_"]::attr(id)').re_first(r'anchor_author_(\d+)')

    result = {
        'name': document.css('h1.title::text').get(),
        'id': tour_id,
        'author_public_name': document.css('div.author a.standard::text').get(),
        'author_internal_name': document.css('img[id^="anchor_author_"]::attr(onmouseover)').re_first(r'"https://www.hikr.org/","\d+","(.*?)","'),
        'author_id': author_id,
        'publishing_date_str': publishing_date_str,
        'publishing_date': publishing_date,
    }
    for attribute in attributes:
        raw_content = document.css(f'td.fiche_rando_b:contains("{attribute}") + td.fiche_rando::text').get()
        if raw_content:
            if raw_content.strip() == '':
                result[f'{attribute}_raw'] = document.css(f'td.fiche_rando_b:contains("{attribute}") + td.fiche_rando').get()
            else:
                result[f'{attribute}_raw'] = raw_content.strip()
        else:
            result[f'{attribute}_raw'] = None
    
    return result

In [4]:
files_dir = "./data/raw/200posts/"
posts_raw = []

# Iterate over all files in the directory
for filename in os.listdir(files_dir):
    if filename.endswith('.html'):
        file_path = os.path.join(files_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        r = parse_raw([f.name, content], unique_attributes)
        posts_raw.append(r)

In [5]:
posts_raw_df = pd.DataFrame(posts_raw)
posts_raw_df

,name,id,author_public_name,author_internal_name,author_id,publishing_date_str,publishing_date,Hochtouren Schwierigkeit:_raw,Ski Schwierigkeit:_raw,Geo-Tags:_raw,...,Zeitbedarf:_raw,Abstieg:_raw,Kartennummer:_raw,Aufstieg:_raw,Klettersteig Schwierigkeit:_raw,Mountainbike Schwierigkeit:_raw,Schneeshuhtouren Schwierigkeit:_raw,Wegpunkte:_raw,Zufahrt zum Ausgangspunkt:_raw,Zufahrt zum Ankunftspunkt:_raw
0,Schlappiner Spitze dans le brouillard,24199,drixdrey,drixdrey,5903,13. Juni 2010 um 19:42,2010-06-13 19:42:00,"<td class=""fiche_rando""><a class=""standard"" href=""https://www.hikr.org/view_post_lang.php?id=240"" target=""help"">\nWS</a>\n</td>",None,"<td class=""fiche_rando""><img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/ch-gr/"">CH-GR</a> \n<img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/a/"">A</a> \n</td>",...,1 Tage 4:30,1300 m,1177 Serneus,550 m,None,None,None,"<td class=""fiche_rando"">\n<ul style=""list-style-type:none;list-style-position:outside"">\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_peak_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Schlappiner_Spitz_19661/"">Schlappiner Spitz 2442 m (5)</a>\n</li>\n\n\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_pass_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Schlappiner_Joch_19660/"">Schlappiner Joch 2203 m (12)</a>\n</li>\n\n\n</ul>\n</td>","Télécabine Madrisa, Klosters Dorf",None
1,"Altenalptürm, Östl. und Mittlerer Turm - anders als geplant (The answer, my friend, is blowing...)",24064,marmotta,marmotta,2891,11. Juni 2010 um 01:06,2010-06-11 01:06:00,None,None,"<td class=""fiche_rando""><img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/alpstein/"">Alpstein</a> \n<img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/ch-ai/"">CH-AI</a> \n<img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/ch-ar/"">CH-AR</a> \n<img alt="""" src=""https://s.hikr.org/r4icons/world.png"" style=""vertical-align:text-bottom;border:0""> <a class=""standard"" href=""https://www.hikr.org/dir/tag/ch-sg/"">CH-SG</a> \n</td>",...,7:00,1500 m,LK 1115 Säntis (1:25.000),2000 m,None,None,None,"<td class=""fiche_rando"">\n<ul style=""list-style-type:none;list-style-position:outside"">\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_ort_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Wasserauen_6854/"">Wasserauen 868 m (381)</a>\n</li>\n\n\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_lake_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Seealpsee_8235/"">Seealpsee 1141 m (217)</a>\n</li>\n\n\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_hut_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Altenalp_10913/"">Altenalp 1595 m (58)</a>\n</li>\n\n\n<li style=""list-style-type:none;list-style-position:outside""><img src=""https://s.hikr.org/r4icons/ico2_peak_s.png"" style=""vertical-align:text-bottom""> <a class=""standard"" href=""https://www.hikr.org/dir/Altenalpt%C3%BCrm_-_Ost_96015/"">Altena

In [6]:
print(posts_raw_df.columns)

Index(['name', 'id', 'author_public_name', 'author_internal_name', 'author_id',
       'publishing_date_str', 'publishing_date',
       'Hochtouren Schwierigkeit:_raw', 'Ski Schwierigkeit:_raw',
       'Geo-Tags:_raw', 'Klettern Schwierigkeit:_raw', 'Region:_raw',
       'Unterkunftmöglichkeiten:_raw', 'Strecke:_raw',
       'Wandern Schwierigkeit:_raw', 'Tour Datum:_raw', 'Zeitbedarf:_raw',
       'Abstieg:_raw', 'Kartennummer:_raw', 'Aufstieg:_raw',
       'Klettersteig Schwierigkeit:_raw', 'Mountainbike Schwierigkeit:_raw',
       'Schneeshuhtouren Schwierigkeit:_raw', 'Wegpunkte:_raw',
       'Zufahrt zum Ausgangspunkt:_raw', 'Zufahrt zum Ankunftspunkt:_raw'],
      dtype='object')


In [7]:
counts = {}

for column in posts_raw_df.columns:
    if column.endswith('_raw'):
        counts[column] = posts_raw_df[column].count()

for column, count in sorted(counts.items(), key=lambda item: item[1], reverse=True):
    print(f'{column}: {count}')

print("----")
print("Total Rows: ", len(posts_raw_df))

Region:_raw: 200
Tour Datum:_raw: 199
Wegpunkte:_raw: 199
Geo-Tags:_raw: 195
Wandern Schwierigkeit:_raw: 169
Aufstieg:_raw: 161
Zeitbedarf:_raw: 158
Abstieg:_raw: 147
Zufahrt zum Ausgangspunkt:_raw: 137
Strecke:_raw: 124
Kartennummer:_raw: 83
Unterkunftmöglichkeiten:_raw: 55
Zufahrt zum Ankunftspunkt:_raw: 50
Klettern Schwierigkeit:_raw: 32
Hochtouren Schwierigkeit:_raw: 10
Mountainbike Schwierigkeit:_raw: 7
Klettersteig Schwierigkeit:_raw: 6
Ski Schwierigkeit:_raw: 5
Schneeshuhtouren Schwierigkeit:_raw: 3
----
Total Rows:  200


In [8]:
from IPython.display import display, HTML
posts_raw_df['Region:_raw'].iloc[0]

# How can I extract all a tags with scrapy?

def parse_region(region_raw: str):
    if region_raw is None:
        return None
    document = Selector(text=region_raw)
    a_tags = document.css('a')
    output = {}
    for i, a_tag in enumerate(a_tags):
        href = a_tag.css('::attr(href)').get()
        content = a_tag.css('::text').get()
        #output[f"region_{i}_href"] = href.strip()
        output[f"region_{i}_content"] = content.strip()
    return output
    

print(parse_region(posts_raw_df['Region:_raw'].iloc[0]))



{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Graubünden', 'region_3_content': 'Prättigau'}


In [9]:
regions = []
for i in range(0, posts_raw_df.shape[0]):
    display(HTML(posts_raw_df['Region:_raw'].iloc[i]))
    region = parse_region(posts_raw_df['Region:_raw'].iloc[i])
    print(region)
    regions.append(region)


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Graubünden', 'region_3_content': 'Prättigau'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Appenzell'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Zürich'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Tessin', 'region_3_content': 'Bellinzonese'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Graubünden', 'region_3_content': 'Prättigau'}


...

{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Bern', 'region_3_content': 'Berner Jura'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Uri'}


{'region_0_content': 'Welt', 'region_1_content': 'Schweiz', 'region_2_content': 'Wallis', 'region_3_content': 'Mittelwallis'}


{'region_0_content': 'Welt', 'region_1_content': 'Frankreich', 'region_2_content': 'Alpes-Maritimes'}


{'region_0_content': 'Welt', 'region_1_content': 'Österreich', 'region_2_content': 'Nördliche Ostalpen', 'region_3_content': 'Rofangebirge und Brandenberger Alpen'}


In [10]:
regions_df = pd.DataFrame(regions)
print(regions_df.columns)

for column in regions_df.columns:
    print(f'{column}: {regions_df[column].count()}')

for column in regions_df.columns:
    print(f'{column}: {regions_df[column].unique()}')    

Index(['region_0_content', 'region_1_content', 'region_2_content',
       'region_3_content'],
      dtype='object')
region_0_content: 200
region_1_content: 200
region_2_content: 191
region_3_content: 93
region_0_content: ['Welt']
region_1_content: ['Schweiz' 'Spanien' 'Deutschland' 'Italien' 'Liechtenstein' 'Frankreich'
 'Österreich' 'Marokko' 'Niederlande' 'Kroatien' 'Panama' 'Ecuador'
 'United Kindom']
region_2_content: ['Graubünden' 'Appenzell' 'Zürich' 'Tessin' 'Kanarische Inseln'
 'St.Gallen' 'Westliche Mittelgebirge' 'Sizilien' 'Glarus' 'Waadt' nan
 'Alpes-Maritimes' 'Nördliche Ostalpen' 'Trentino-Südtirol' 'Wallis' 'Uri'
 'Schwyz' 'Obwalden' 'Bern' 'Zentrale Ostalpen' 'Aargau' 'Vogesen'
 'Freiburg' 'Alpenvorland' 'Alpen' 'Lombardei' 'Piemont' "Cote d'Azur"
 'Schaffhausen' 'Nidwalden' 'Venetien' 'Südwestliche Mittelgebirge'
 'Solothurn' 'Aostatal' 'Luzern' 'Haute-Savoie' 'Thurgau' 'Kampanien'
 'Schottland' 'Neuenburg']
region_3_content: ['Prättigau' nan 'Bellinzonese' 'Santa Cru

In [11]:
print(posts_raw_df['Tour Datum:_raw'])

0      13 Juni 2010
1      10 Juni 2010
2      13 Juni 2010
3       3 Juni 2010
4      12 Juni 2010
           ...     
195    12 Juni 2010
196    12 Juni 2010
197    15 Juni 2010
198     3 Juni 2010
199    20 Juli 2009
Name: Tour Datum:_raw, Length: 200, dtype: object


In [12]:
def parse_tour_date(tour_date_raw: str):
    if tour_date_raw is None:
        return None
    
    return datetime.strptime(tour_date_raw, '%d %B %Y').date()

print(parse_tour_date(posts_raw_df['Tour Datum:_raw'].iloc[0]))

2010-06-13


In [13]:
# Parse the tour date for all rows
tour_dates = []
for i in range(0, posts_raw_df.shape[0]):
    raw = posts_raw_df['Tour Datum:_raw'].iloc[i]
    tour_date = parse_tour_date(raw)
    tour_dates.append(tour_date)
    print(f"Raw: {raw}, Parsed: {tour_date}")



Raw: 13 Juni 2010, Parsed: 2010-06-13
Raw: 10 Juni 2010, Parsed: 2010-06-10
Raw: 13 Juni 2010, Parsed: 2010-06-13
Raw: 3 Juni 2010, Parsed: 2010-06-03
Raw: 12 Juni 2010, Parsed: 2010-06-12
Raw: 31 Mai 2010, Parsed: 2010-05-31
Raw: 7 Juli 2009, Parsed: 2009-07-07
Raw: 25 März 2003, Parsed: 2003-03-25
Raw: 9 Juni 2010, Parsed: 2010-06-09
Raw: 10 Juni 2010, Parsed: 2010-06-10
Raw: 29 Mai 2010, Parsed: 2010-05-29
Raw: 13 Juni 2010, Parsed: 2010-06-13
Raw: 15 November 2008, Parsed: 2008-11-15
Raw: 10 Juni 2010, Parsed: 2010-06-10
Raw: 29 Mai 2010, Parsed: 2010-05-29
Raw: 24 April 2010, Parsed: 2010-04-24
Raw: 2 Juni 2010, Parsed: 2010-06-02
Raw: 6 Juni 2010, Parsed: 2010-06-06
Raw: 5 Juni 2010, Parsed: 2010-06-05
Raw: 4 Juni 2010, Parsed: 2010-06-04
Raw: 22 Juni 2010, Parsed: 2010-06-22
Raw: 5 Dezember 2004, Parsed: 2004-12-05
Raw: 9 Juni 2010, Parsed: 2010-06-09
Raw: 5 Juni 2010, Parsed: 2010-06-05
Raw: 5 Juni 2010, Parsed: 2010-06-05
Raw: 3 Juni 2012, Parsed: 2012-06-03
Raw: 12 Juni 2010,

In [14]:
print(posts_raw_df['Wegpunkte:_raw'].iloc[1])
display(HTML(posts_raw_df['Wegpunkte:_raw'].iloc[1]))

<td class="fiche_rando">
<ul style="list-style-type:none;list-style-position:outside">
<li style="list-style-type:none;list-style-position:outside"><img src="https://s.hikr.org/r4icons/ico2_ort_s.png" style="vertical-align:text-bottom"> <a class="standard" href="https://www.hikr.org/dir/Wasserauen_6854/">Wasserauen 868 m   (381)</a>
</li>


<li style="list-style-type:none;list-style-position:outside"><img src="https://s.hikr.org/r4icons/ico2_lake_s.png" style="vertical-align:text-bottom"> <a class="standard" href="https://www.hikr.org/dir/Seealpsee_8235/">Seealpsee 1141 m   (217)</a>
</li>


<li style="list-style-type:none;list-style-position:outside"><img src="https://s.hikr.org/r4icons/ico2_hut_s.png" style="vertical-align:text-bottom"> <a class="standard" href="https://www.hikr.org/dir/Altenalp_10913/">Altenalp 1595 m   (58)</a>
</li>


<li style="list-style-type:none;list-style-position:outside"><img src="https://s.hikr.org/r4icons/ico2_peak_s.png" style="vertical-align:text-bottom

In [15]:
# ALL options can be found on the hikr.org website in the html file during route creation: https://www.hikr.org/add_rando.php#fiche
# No need to save anything
icon_lookup = {
    "https://s.hikr.org/r4icons/ico2_point_s.png": "other",
    "https://s.hikr.org/r4icons/ico2_ruin_s.png": "ruin",
    "https://s.hikr.org/r4icons/ico2_cave_s.png": "cave",
    "https://s.hikr.org/r4icons/ico2_ort_s.png": "location",
    "https://s.hikr.org/r4icons/ico2_bridg_s.png": "bridge",
    "https://s.hikr.org/r4icons/ico2_pass_s.png": "pass",
    "https://s.hikr.org/r4icons/ico2_viafe_s.png": "via_ferrata", #Klettersteig
    "https://s.hikr.org/r4icons/ico2_climb_s.png": "climb",
    "https://s.hikr.org/r4icons/ico2_wand_s.png": "wall",
    "https://s.hikr.org/r4icons/ico2_lake_s.png": "lake",
    "https://s.hikr.org/r4icons/ico2_peak_s.png": "peak",
    "https://s.hikr.org/r4icons/ico2_hut_s.png": "hut",
    "https://s.hikr.org/r4icons/ico2_eisfa_s.png": "icefall",
}



def parse_waypoints(wegpunkte_raw: str):
    if wegpunkte_raw is None:
        return None
    document = Selector(text=wegpunkte_raw)
    list_items = document.css('li:not([class])')
    # This is a special casse for some of the waypoints also have subpoints which I ignore for now: example: post24156.html

    
    output = []
    for i, li in enumerate(list_items):
        image = li.css('img::attr(src)').get()
        name_raw = li.css('a::text').get().strip()
        waypoint_url = li.css('a::attr(href)').get()
        peak_id = li.css('a::attr(href)').re_first(r'(\d+)')
        
        regex_pattern = r"(.*?)\s+(\d+)\s+m" # https://regex101.com/r/MNWWxW/1


        if image is None:
            print(f"Image not found for {name_raw}")
        match = re.search(regex_pattern, name_raw)
        if match:
            name = match.group(1).strip()
            height = int(match.group(2))
        waypoint = {
            'image': image,
            'name_raw': name_raw,
            'type': icon_lookup.get(image, None),
            'waypoint_url': waypoint_url,
            'height': height,
            'name': name,
            'peak_id': peak_id
        }
        output.append(waypoint)
    return output


parse_waypoints(posts_raw_df['Wegpunkte:_raw'].iloc[1])

[{'image': 'https://s.hikr.org/r4icons/ico2_ort_s.png',
  'name_raw': 'Wasserauen 868 m   (381)',
  'type': 'location',
  'waypoint_url': 'https://www.hikr.org/dir/Wasserauen_6854/',
  'height': 868,
  'name': 'Wasserauen',
  'peak_id': '6854'},
 {'image': 'https://s.hikr.org/r4icons/ico2_lake_s.png',
  'name_raw': 'Seealpsee 1141 m   (217)',
  'type': 'lake',
  'waypoint_url': 'https://www.hikr.org/dir/Seealpsee_8235/',
  'height': 1141,
  'name': 'Seealpsee',
  'peak_id': '8235'},
 {'image': 'https://s.hikr.org/r4icons/ico2_hut_s.png',
  'name_raw': 'Altenalp 1595 m   (58)',
  'type': 'hut',
  'waypoint_url': 'https://www.hikr.org/dir/Altenalp_10913/',
  'height': 1595,
  'name': 'Altenalp',
  'peak_id': '10913'},
 {'image': 'https://s.hikr.org/r4icons/ico2_peak_s.png',
  'name_raw': 'Altenalptürm - Ost 1986 m   (11)',
  'type': 'peak',
  'waypoint_url': 'https://www.hikr.org/dir/Altenalpt%C3%BCrm_-_Ost_96015/',
  'height': 1986,
  'name': 'Altenalptürm - Ost',
  'peak_id': '3'},
 {'

In [16]:
# Parse the waypoints for all rows
waypoints = []
for i in range(0, posts_raw_df.shape[0]):
    raw = posts_raw_df['Wegpunkte:_raw'].iloc[i]
    waypoints_data = parse_waypoints(raw)
    waypoints.append(waypoints_data)

print(waypoints)

[[{'image': 'https://s.hikr.org/r4icons/ico2_peak_s.png', 'name_raw': 'Schlappiner Spitz 2442 m   (5)', 'type': 'peak', 'waypoint_url': 'https://www.hikr.org/dir/Schlappiner_Spitz_19661/', 'height': 2442, 'name': 'Schlappiner Spitz', 'peak_id': '19661'}, {'image': 'https://s.hikr.org/r4icons/ico2_pass_s.png', 'name_raw': 'Schlappiner Joch 2203 m   (12)', 'type': 'pass', 'waypoint_url': 'https://www.hikr.org/dir/Schlappiner_Joch_19660/', 'height': 2203, 'name': 'Schlappiner Joch', 'peak_id': '19660'}], [{'image': 'https://s.hikr.org/r4icons/ico2_ort_s.png', 'name_raw': 'Wasserauen 868 m   (381)', 'type': 'location', 'waypoint_url': 'https://www.hikr.org/dir/Wasserauen_6854/', 'height': 868, 'name': 'Wasserauen', 'peak_id': '6854'}, {'image': 'https://s.hikr.org/r4icons/ico2_lake_s.png', 'name_raw': 'Seealpsee 1141 m   (217)', 'type': 'lake', 'waypoint_url': 'https://www.hikr.org/dir/Seealpsee_8235/', 'height': 1141, 'name': 'Seealpsee', 'peak_id': '8235'}, {'image': 'https://s.hikr.org/

In [17]:
# Collect all uniwue immages
unique_images = set()

for i in waypoints:
    if i is None:
        continue
    for waypoint in i:
        unique_images.add(waypoint['image'])

for image in unique_images:
    display(HTML(f'<div><img src="{image}" /> {image}<div>'))
    



In [18]:
unique_names = set()
for i in waypoints:
    if i is None:
        continue
    for waypoint in i:
        unique_names.add(waypoint['name_raw'])

for name in unique_names:
    print(name)

Rifugio Corte Nuovo 1635 m   (32)
Mandli 2007 m   (12)
Jetzmunt 845 m   (4)
Schmidshof 526 m   (1)
Goldau 510 m   (59)
Marwees - Ostgipfel 1991 m   (102)
Oropa - parcheggio funivia 1221 m   (19)
Puerto Naos 10 m   (2)
Eggerberg 846 m   (55)
Seebergspitze 2085 m   (21)
Volcán Deseada 1949 m   (12)
Bärnet 2385 m   (7)
Larèchia 1596 m   (11)
Schöllenen 1399 m   (24)
Grosser Mythen 1898 m   (225)
Somada Alta 1947 m   (2)
Andermatt 1436 m   (162)
Kulm 1433 m   (30)
Laghi Campelli 2036 m   (10)
Harder Kulm 1322 m   (61)
Plan de Châtel 1474 m   (3)
Pico Bejenado 1856 m   (8)
Burg Katz 130 m   (4)
Mittler Gassenstock 2275 m   (2)
Saaspass 1896 m   (38)
Bütziflue / Bützi 917 m   (110)
Bernina Suot 2046 m   (32)
Königssee 610 m   (37)
Vent 1896 m   (145)
Rigi Kulm 1798 m   (206)
Sigbachfall 770 m   (20)
Abbazia San Benedetto in Val Perlana 825 m   (16)
Bassa di Söu 2443 m   (40)
Eus 1603 m   (28)
Schaan 457 m   (15)
Vättis 943 m   (66)
Gadon 690 m   (1)
Zingel 1525 m   (16)
Monte Covreto 1594 m 

In [19]:

for i in waypoints:
    if i is None:
        continue
    for waypoint in i:
        print(f"Name: {waypoint['name']}, Type: {waypoint['type']}, Height: {waypoint['height']}")


Name: Schlappiner Spitz, Type: peak, Height: 2442
Name: Schlappiner Joch, Type: pass, Height: 2203
Name: Wasserauen, Type: location, Height: 868
Name: Seealpsee, Type: lake, Height: 1141
Name: Altenalp, Type: hut, Height: 1595
Name: Altenalptürm - Ost, Type: peak, Height: 1986
Name: Altenalptürm, Type: peak, Height: 2031
Name: Öhrlisattel, Type: pass, Height: 2121
Name: Öhrli / Öhrlikopf, Type: peak, Height: 2193
Name: Hängeten - Westlicher Vorgipfel, Type: peak, Height: 2205
Name: Schwägalp, Type: location, Height: 1352
Name: Zürich - Haltestelle Triemli, Type: other, Height: 449
Name: Ruine Friesenburg, Type: ruin, Height: 658
Name: Üetliberg - Aussichtsturm, Type: other, Height: 900
Name: Cima di Biasagn, Type: peak, Height: 2417
Name: Capanna Biasagn, Type: hut, Height: 2023
Name: Fontana, Type: other, Height: 1347
Name: Riedchopf / Ronggspitz, Type: peak, Height: 2552
Name: Vilaflor, Type: location, Height: 1420
Name: Paisaje Lunar, Type: other, Height: 1750
Name: Montaña de Guaja

In [20]:
# Wandern Schwierigkeit:_raw:
print(posts_raw_df['Wandern Schwierigkeit:_raw'].iloc[0])
display(HTML(posts_raw_df['Wandern Schwierigkeit:_raw'].iloc[0]))

<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=238" target="help">
T4 - Alpinwandern
</a>
</td>


In [21]:
def parse_hiking_difficulty(hiking_difficulty_raw: str):
    if hiking_difficulty_raw is None:
        return None
    document = Selector(text=hiking_difficulty_raw)
    regex_pattern = r"(T\d[+-]?)\s*-\s*(.*)" #https://regex101.com/r/otbIAQ/1
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    match = re.search(regex_pattern, a_tags[0].css('::text').get().strip())
    if match:
        return {
            "difficulty": match.group(1),
            "description": match.group(2)
        }
    return  None


In [22]:
unique_hiking_difficulties = set()
for i in range(0, posts_raw_df.shape[0]):
    hiking_difficulty = parse_hiking_difficulty(posts_raw_df['Wandern Schwierigkeit:_raw'].iloc[i])
    if hiking_difficulty is not None:
        unique_hiking_difficulties.add(hiking_difficulty['difficulty'])


print(f'Number of unique hiking difficulties: {len(unique_hiking_difficulties)}')
for difficulty in sorted(unique_hiking_difficulties):
    print(difficulty)

Number of unique hiking difficulties: 12
T1
T2
T3
T3+
T4
T4+
T4-
T5
T5+
T6
T6+
T6-


In [23]:
# Aufstieg
for i in range(0, posts_raw_df.shape[0]):
    print(posts_raw_df['Aufstieg:_raw'].iloc[i])

def parse_ascent(ascent_raw: str) -> int:
    if ascent_raw is None:
        return None
    return int(ascent_raw.split(' ')[0].strip())

550 m
2000 m
None
1050 m
1100 m
1300 m
1385 m
400 m
2000 m
580 m
350 m
529 m
1098 m
320 m
1600 m
1080 m
800 m
1800 m
None
1300 m
800 m
1200 m
1700 m
None
1500 m
None
970 m
1280 m
600 m
1100 m
150 m
1580 m
None
300 m
None
950 m
450 m
330 m
1280 m
800 m
1100 m
1250 m
None
610 m
None
1310 m
742 m
1300 m
1050 m
420 m
600 m
None
200 m
400 m
900 m
1600 m
942 m
1730 m
744 m
2100 m
770 m
1400 m
420 m
150 m
None
1800 m
400 m
776 m
None
70 m
1100 m
None
780 m
760 m
None
None
60 m
1200 m
None
None
1150 m
None
1500 m
1150 m
590 m
1600 m
2305 m
1100 m
630 m
640 m
2900 m
1500 m
None
1070 m
1450 m
335 m
1200 m
None
1150 m
400 m
1250 m
2720 m
1602 m
500 m
1230 m
None
720 m
5 m
1200 m
600 m
2000 m
None
450 m
400 m
600 m
1370 m
2150 m
1660 m
70 m
1200 m
1500 m
None
None
330 m
1100 m
1000 m
589 m
310 m
650 m
None
750 m
180 m
None
1300 m
None
400 m
770 m
1093 m
None
None
820 m
500 m
1230 m
640 m
None
1500 m
1100 m
None
2060 m
1300 m
200 m
550 m
700 m
None
None
750 m
700 m
1150 m
None
None
1000 m
None
900 

In [24]:
for i in range(0, posts_raw_df.shape[0]):
    ascent = parse_ascent(posts_raw_df['Aufstieg:_raw'].iloc[i])
    print(ascent)

550
2000
None
1050
1100
1300
1385
400
2000
580
350
529
1098
320
1600
1080
800
1800
None
1300
800
1200
1700
None
1500
None
970
1280
600
1100
150
1580
None
300
None
950
450
330
1280
800
1100
1250
None
610
None
1310
742
1300
1050
420
600
None
200
400
900
1600
942
1730
744
2100
770
1400
420
150
None
1800
400
776
None
70
1100
None
780
760
None
None
60
1200
None
None
1150
None
1500
1150
590
1600
2305
1100
630
640
2900
1500
None
1070
1450
335
1200
None
1150
400
1250
2720
1602
500
1230
None
720
5
1200
600
2000
None
450
400
600
1370
2150
1660
70
1200
1500
None
None
330
1100
1000
589
310
650
None
750
180
None
1300
None
400
770
1093
None
None
820
500
1230
640
None
1500
1100
None
2060
1300
200
550
700
None
None
750
700
1150
None
None
1000
None
900
450
2000
1200
250
1100
1060
1430
1370
750
700
715
360
1077
100
1500
1500
400
300
900
None
1050
1400
900
725
1400
None
1060
1471
1350
1500
1400
650
None
None
450
900
680


In [25]:
# Zeitbedarf
for i in range(0, posts_raw_df.shape[0]):
    print(posts_raw_df['Zeitbedarf:_raw'].iloc[i])

1 Tage 4:30
7:00
3:00
2 Tage 7:00
1 Tage 5:00
None
3:00
2:30
None
4:30
6:00
4:00
None
None
2 Tage
4:15
4:30
9:00
None
None
4:00
None
8:00
None
6:00
None
5:00
4:00
4:30
7:00
1:30
3:15
5:00
3:30
None
None
5 Tage
2:30
5:30
4:30
3:30
1 Tage 6:00
None
4:00
5:00
None
5:00
5:30
5:30
1:45
4:30
None
2:00
3:00
5:00
None
4:30
1 Tage
2:30
None
3:30
7:00
4:00
1:30
None
None
3:30
7:45
None
0:45
7:30
4:30
1 Tage 4:00
None
5:00
None
None
5:00
7:00
7:00
6:00
None
9:00
None
1 Tage 6:00
8:30
16:30
5:15
1 Tage
3:30
14:00
None
3:15
8:45
5:30
6:45
5:00
4:00
3:45
2:00
5:00
10:00
10:00
2:00
4 Tage
None
5:00
2:15
4:00
1:30
6:00
3:00
None
3:30
3:00
2 Tage
7:00
7:00
0:30
2 Tage
None
2:00
5:00
4:00
6:30
6:00
3:00
1:00
1 Tage 4:00
8:00
4:30
5:30
None
4:30
None
2:30
None
1 Tage
3:00
13:00
5:00
3:00
5:30
3:15
None
6:45
5:30
None
4:30
7:00
7:30
3:00
2:00
None
None
5:45
None
6:30
None
None
2:30
1 Tage
3:30
2:15
7:00
None
2:45
7:00
3:15
4:00
5:30
3:00
3:30
5:00
4:00
4:30
2:00
6:45
5:00
4:00
2:30
5:00
8 Tage
3:30
4:00
4

In [26]:
def parse_duration(duration_raw: str):
    if duration_raw is None:
        return None
    regex_pattern = re.compile(r"(\d+)?\s?(?:Tage|days)?\s?(\d{1,2}(?::\d{2}))?")  #https://regex101.com/r/rVlCVe/1
    match = re.search(regex_pattern, duration_raw)
    if match:
        days = int(match.group(1)) if match.group(1) else 0
        hours_minutes = match.group(2) if match.group(2) else "0:00"
        hours, minutes = map(int, hours_minutes.split(":"))
        
        return timedelta(days=days, hours=hours, minutes=minutes)
        
    return None

In [27]:
for i in range(0, posts_raw_df.shape[0]):
    duration = parse_duration(posts_raw_df['Zeitbedarf:_raw'].iloc[i])
    print(duration)

1 day, 4:30:00
7 days, 0:00:00
3 days, 0:00:00
2 days, 7:00:00
1 day, 5:00:00
None
3 days, 0:00:00
2 days, 0:00:00
None
4 days, 0:00:00
6 days, 0:00:00
4 days, 0:00:00
None
None
2 days, 0:00:00
4 days, 0:00:00
4 days, 0:00:00
9 days, 0:00:00
None
None
4 days, 0:00:00
None
8 days, 0:00:00
None
6 days, 0:00:00
None
5 days, 0:00:00
4 days, 0:00:00
4 days, 0:00:00
7 days, 0:00:00
1 day, 0:00:00
3 days, 0:00:00
5 days, 0:00:00
3 days, 0:00:00
None
None
5 days, 0:00:00
2 days, 0:00:00
5 days, 0:00:00
4 days, 0:00:00
3 days, 0:00:00
1 day, 6:00:00
None
4 days, 0:00:00
5 days, 0:00:00
None
5 days, 0:00:00
5 days, 0:00:00
5 days, 0:00:00
1 day, 0:00:00
4 days, 0:00:00
None
2 days, 0:00:00
3 days, 0:00:00
5 days, 0:00:00
None
4 days, 0:00:00
1 day, 0:00:00
2 days, 0:00:00
None
3 days, 0:00:00
7 days, 0:00:00
4 days, 0:00:00
1 day, 0:00:00
None
None
3 days, 0:00:00
7 days, 0:00:00
None
0:00:00
7 days, 0:00:00
4 days, 0:00:00
1 day, 4:00:00
None
5 days, 0:00:00
None
None
5 days, 0:00:00
7 days, 0:

In [28]:
# Abstieg
for i in range(0, posts_raw_df.shape[0]):
    print(posts_raw_df['Abstieg:_raw'].iloc[i])



1300 m
1500 m
None
1050 m
1100 m
570 m
1385 m
400 m
2000 m
580 m
750 m
None
1098 m
270 m
1900 m
1080 m
800 m
1800 m
None
1300 m
250 m
None
1700 m
None
1500 m
None
970 m
350 m
600 m
1100 m
150 m
245 m
None
800 m
None
950 m
None
330 m
1280 m
800 m
1100 m
550 m
None
570 m
None
1310 m
742 m
1300 m
1050 m
420 m
600 m
None
200 m
400 m
900 m
1600 m
942 m
1730 m
744 m
2100 m
770 m
1400 m
420 m
150 m
None
1800 m
400 m
None
None
70 m
1100 m
None
780 m
760 m
None
None
60 m
1200 m
None
None
1150 m
None
1100 m
1150 m
590 m
1600 m
2305 m
1100 m
1630 m
640 m
None
1500 m
None
1070 m
1450 m
1270 m
1200 m
None
1150 m
400 m
1250 m
2620 m
1553 m
200 m
500 m
None
800 m
5 m
1200 m
600 m
2000 m
None
450 m
400 m
600 m
None
2150 m
None
None
1200 m
None
None
None
750 m
200 m
1300 m
659 m
310 m
650 m
None
750 m
180 m
None
1300 m
None
100 m
770 m
1100 m
None
None
820 m
500 m
1230 m
640 m
None
1500 m
1100 m
None
None
1300 m
170 m
550 m
700 m
None
None
750 m
700 m
1150 m
None
None
250 m
None
900 m
450 m
600 m
1200 

In [29]:
def parse_descent(descent_raw: str) -> int:
    if descent_raw is None:
        return None
    return int(descent_raw.split(' ')[0].strip())


for i in range(0, posts_raw_df.shape[0]):
    descent = parse_descent(posts_raw_df['Abstieg:_raw'].iloc[i])
    print(descent)

1300
1500
None
1050
1100
570
1385
400
2000
580
750
None
1098
270
1900
1080
800
1800
None
1300
250
None
1700
None
1500
None
970
350
600
1100
150
245
None
800
None
950
None
330
1280
800
1100
550
None
570
None
1310
742
1300
1050
420
600
None
200
400
900
1600
942
1730
744
2100
770
1400
420
150
None
1800
400
None
None
70
1100
None
780
760
None
None
60
1200
None
None
1150
None
1100
1150
590
1600
2305
1100
1630
640
None
1500
None
1070
1450
1270
1200
None
1150
400
1250
2620
1553
200
500
None
800
5
1200
600
2000
None
450
400
600
None
2150
None
None
1200
None
None
None
750
200
1300
659
310
650
None
750
180
None
1300
None
100
770
1100
None
None
820
500
1230
640
None
1500
1100
None
None
1300
170
550
700
None
None
750
700
1150
None
None
250
None
900
450
600
1200
250
1100
None
None
1370
None
700
715
360
1077
127
1500
None
400
300
900
None
1050
700
900
740
1400
None
1060
1471
1350
1500
1400
650
None
None
430
900
680


In [30]:
# Zufahrt zum Ausgangspunkt:_raw:

print(posts_raw_df['Zufahrt zum Ausgangspunkt:_raw'].iloc[1])


## Currenty not parsin because it is a unstructed freetext field

<td class="fiche_rando"><img src="https://s.hikr.org/r4icons/cff.gif" style="border:0;vertical-align:middle;" alt="cff logo"> <a rel="nofollow" href="http://www.cff.ch/" onclick="r4showOVForm('Wasserauen');return false;">Wasserauen</a><br></td>


In [31]:
# Zufahrt zum Strecke:_raw:

print(posts_raw_df['Strecke:_raw'].iloc[6])

Dirinella (210)- Scaiano- Monti di Caviano (695)- Alpetto di Caviano (1255)- Sasso Torricello (1424)- Covreto (1595) - ritorno dallo stesso sentiero


In [32]:
# Kartennummer:_raw
print(posts_raw_df['Kartennummer:_raw'].iloc[1])
# I don't see any value in parsing this field

LK 1115 Säntis (1:25.000)


In [33]:
#Klettern Schwierigkeit:_raw
print(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[1])
display(HTML(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[1]))

for i in range(0, 10):
    print(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[i])
    display(HTML(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[i]))

<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=240" target="help">
II<span style="color:#888;text-decoration:none;background:#fff;"> (UIAA-Skala)</span></a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=240" target="help">
II<span style="color:#888;text-decoration:none;background:#fff;"> (UIAA-Skala)</span></a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=240" target="help">
II<span style="color:#888;text-decoration:none;background:#fff;"> (UIAA-Skala)</span></a>
</td>


None


<IPython.core.display.HTML object>

...

None


<IPython.core.display.HTML object>

<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=240" target="help">
II<span style="color:#888;text-decoration:none;background:#fff;"> (UIAA-Skala)</span></a>
</td>


None


<IPython.core.display.HTML object>

In [34]:
def parse_climbing_difficulty(climbing_difficulty_raw: str):
    if climbing_difficulty_raw is None:
        return None
    document = Selector(text=climbing_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    return {
        "climbing_difficulty": a_tags[0].css('::text').get().strip(),
        "climbing_scale": a_tags[0].css('span::text').get().strip()
    }

In [35]:
parse_climbing_difficulty(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[1])

{'climbing_difficulty': 'II', 'climbing_scale': '(UIAA-Skala)'}

In [36]:
unique_climbing_difficulties = set()
uniuqe_climbing_scales = set()
for i in range(0, posts_raw_df.shape[0]):
    climbing_difficulty = parse_climbing_difficulty(posts_raw_df['Klettern Schwierigkeit:_raw'].iloc[i])
    if climbing_difficulty is not None:
        unique_climbing_difficulties.add(climbing_difficulty['climbing_difficulty'])
        uniuqe_climbing_scales.add(climbing_difficulty['climbing_scale'])



print(f'Number of unique climbing difficulties: {len(unique_climbing_difficulties)}')
for difficulty in sorted(unique_climbing_difficulties):
    print(difficulty)

print(f'Number of unique climbing scales: {len(uniuqe_climbing_scales)}')
for scale in sorted(uniuqe_climbing_scales):
    print(scale)

Number of unique climbing difficulties: 11
5b
5c
6a
6a+
I
II
III
IV
V
V+
VII+
Number of unique climbing scales: 2
(Französische Skala)
(UIAA-Skala)


In [37]:
# Hochtouren Schwierigkeit:_raw
print(posts_raw_df['Hochtouren Schwierigkeit:_raw'].iloc[0])

<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=240" target="help">
WS</a>
</td>


In [38]:
def parse_high_tour_difficulty(high_tour_difficulty_raw: str):
    if high_tour_difficulty_raw is None:
        return None
    document = Selector(text=high_tour_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    return a_tags[0].css('::text').get().strip()

In [39]:
parse_high_tour_difficulty(posts_raw_df['Hochtouren Schwierigkeit:_raw'].iloc[0])

unique_high_tour_difficulties = set()
for i in range(0, posts_raw_df.shape[0]):
    high_tour_difficulty = parse_high_tour_difficulty(posts_raw_df['Hochtouren Schwierigkeit:_raw'].iloc[i])
    if high_tour_difficulty is not None:
        unique_high_tour_difficulties.add(high_tour_difficulty)

print(f'Number of unique high tour difficulties: {len(unique_high_tour_difficulties)}')
for difficulty in sorted(unique_high_tour_difficulties):
    print(difficulty)

Number of unique high tour difficulties: 5
L
WS
WS+
WS-
ZS


In [40]:
# Mountainbike Schwierigkeit:_raw:
# print first not null value
for i in range(0, posts_raw_df.shape[0]):
    if posts_raw_df['Mountainbike Schwierigkeit:_raw'].iloc[i] is not None:
        print(posts_raw_df['Mountainbike Schwierigkeit:_raw'].iloc[i])
        break

<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=12351" target="help">
L - Leicht fahrbar                                                                                                                                           
</a>
</td>


In [41]:
def parse_mountainbike_difficulty(mountainbike_difficulty_raw: str):
    if mountainbike_difficulty_raw is None:
        return None
    document = Selector(text=mountainbike_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    return a_tags[0].css('::text').get().strip()

In [42]:
unique_mountainbike_difficulties = set()
for i in range(0, posts_raw_df.shape[0]):
    if posts_raw_df['Mountainbike Schwierigkeit:_raw'].iloc[i] is not None:
        display(HTML(posts_raw_df['Mountainbike Schwierigkeit:_raw'].iloc[i]))
for i in range(0, posts_raw_df.shape[0]):
    mountainbike_difficulty = parse_mountainbike_difficulty(posts_raw_df['Mountainbike Schwierigkeit:_raw'].iloc[i])
    if mountainbike_difficulty is not None:
        unique_mountainbike_difficulties.add(mountainbike_difficulty)

print(f'Number of unique mountainbike difficulties: {len(unique_mountainbike_difficulties)}')
for difficulty in sorted(unique_mountainbike_difficulties):
    print(difficulty)


Number of unique mountainbike difficulties: 2
L - Leicht fahrbar
WS - Gut fahrbar


In [43]:
#Klettersteig Schwierigkeit:_raw:
for i in range(0, posts_raw_df.shape[0]):
    if posts_raw_df['Klettersteig Schwierigkeit:_raw'].iloc[i] is not None:
        print(posts_raw_df['Klettersteig Schwierigkeit:_raw'].iloc[i])
        display(HTML(posts_raw_df['Klettersteig Schwierigkeit:_raw'].iloc[i]))



<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
S</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
WS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
L</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
ZS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
L</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=292" target="help">
L</a>
</td>


In [44]:
def parse_via_ferrata_difficulty(via_ferrata_difficulty_raw: str):
    if via_ferrata_difficulty_raw is None:
        return None
    document = Selector(text=via_ferrata_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    return a_tags[0].css('::text').get().strip()

In [45]:
unique_via_ferrata_difficulties = set()
for i in range(0, posts_raw_df.shape[0]):
    via_ferrata_difficulty = parse_via_ferrata_difficulty(posts_raw_df['Klettersteig Schwierigkeit:_raw'].iloc[i])
    if via_ferrata_difficulty is not None:
        unique_via_ferrata_difficulties.add(via_ferrata_difficulty)

print(f'Number of unique via ferrata difficulties: {len(unique_via_ferrata_difficulties)}')
for difficulty in sorted(unique_via_ferrata_difficulties):
    print(difficulty)

Number of unique via ferrata difficulties: 4
L
S
WS
ZS


In [46]:
#Ski Schwierigkeit:_raw:
for i in range(0, posts_raw_df.shape[0]):
    if posts_raw_df['Ski Schwierigkeit:_raw'].iloc[i] is not None:
        print(posts_raw_df['Ski Schwierigkeit:_raw'].iloc[i])
        display(HTML(posts_raw_df['Ski Schwierigkeit:_raw'].iloc[i]))


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=249" target="help">
WS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=249" target="help">
WS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=249" target="help">
WS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=249" target="help">
WS</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=249" target="help">
ZS</a>
</td>


In [47]:
def parse_ski_difficulty(ski_difficulty_raw: str):
    if ski_difficulty_raw is None:
        return None
    document = Selector(text=ski_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    return a_tags[0].css('::text').get().strip()

In [48]:
unique_ski_difficulties = set()
for i in range(0, posts_raw_df.shape[0]):
    ski_difficulty = parse_ski_difficulty(posts_raw_df['Ski Schwierigkeit:_raw'].iloc[i])
    if ski_difficulty is not None:
        unique_ski_difficulties.add(ski_difficulty)

print(f'Number of unique ski difficulties: {len(unique_ski_difficulties)}')
for difficulty in sorted(unique_ski_difficulties):
    print(difficulty)

Number of unique ski difficulties: 2
WS
ZS


In [49]:
#Schneeshuhtouren Schwierigkeit:_raw:
for i in range(0, posts_raw_df.shape[0]):
    if posts_raw_df['Schneeshuhtouren Schwierigkeit:_raw'].iloc[i] is not None:
        print(posts_raw_df['Schneeshuhtouren Schwierigkeit:_raw'].iloc[i])
        display(HTML(posts_raw_df['Schneeshuhtouren Schwierigkeit:_raw'].iloc[i]))


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=241" target="help">
WT3 - 
 Anspruchsvolle Schneeschuhwanderung
</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=241" target="help">
WT3 - 
 Anspruchsvolle Schneeschuhwanderung
</a>
</td>


<td class="fiche_rando"><a class="standard" href="https://www.hikr.org/view_post_lang.php?id=241" target="help">
WT4 - 
 Schneeschuhtour
</a>
</td>


In [50]:
def parse_snowshoe_tour_difficulty(snowshoe_tour_difficulty_raw: str):
    if snowshoe_tour_difficulty_raw is None:
        return None
    document = Selector(text=snowshoe_tour_difficulty_raw)
    a_tags = document.css('a')
    assert len(a_tags) == 1, "Several A tags"
    text = a_tags[0].css('::text').get().strip()
    text = text.split(' - ')
    return {
        "snowshoe_difficulty": text[0].strip(),
        "snowshoe_description": text[1].strip()
    }

In [51]:
unique_snowshoe_tour_difficulties = set()
unique_snowshoe_tour_descriptions = set()
for i in range(0, posts_raw_df.shape[0]):
    snowshoe_tour_difficulty = parse_snowshoe_tour_difficulty(posts_raw_df['Schneeshuhtouren Schwierigkeit:_raw'].iloc[i])
    if snowshoe_tour_difficulty is not None:
        unique_snowshoe_tour_difficulties.add(snowshoe_tour_difficulty['snowshoe_difficulty'])
        unique_snowshoe_tour_descriptions.add(snowshoe_tour_difficulty['snowshoe_description'])

print(f'Number of unique snowshoe tour difficulties: {len(unique_snowshoe_tour_difficulties)}')
for difficulty in sorted(unique_snowshoe_tour_difficulties):
    print(difficulty)

print(f'Number of unique snowshoe tour descriptions: {len(unique_snowshoe_tour_descriptions)}')
for description in sorted(unique_snowshoe_tour_descriptions):
    print(description)

Number of unique snowshoe tour difficulties: 2
WT3
WT4
Number of unique snowshoe tour descriptions: 2
Anspruchsvolle Schneeschuhwanderung
Schneeschuhtour


## Extract some other features from the HTML files

In [52]:
complete_file_content = []

In [53]:
for filename in os.listdir(files_dir):
    if filename.endswith('.html'):
        print(filename)
        file_path = os.path.join(files_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            complete_file_content.append(f.read())
        break

post24199.html


In [54]:
complete_file_content[0]

def count_photos(content):
    # count photo_id's
    photo_ids = re.findall(r'photo_id:(\d+)', content)
    return len(photo_ids)

count_photos(complete_file_content[0])
    


12

In [55]:
def parse_peak_map(html_content: str):
    pattern = r'pizs\.push\(\{.*?piz_lat:([\d.]+),.*?piz_lon:([\d.]+),.*?piz_name:"(.*?)",.*?piz_height:(\d+),.*?piz_id:(\d+).*?\}\)' ## Regex: https://regex101.com/r/3EiAxt/1
    matches = re.findall(pattern, html_content, re.DOTALL)
        
     # Parse the peak information and yield the data
    peaks = []
    for match in matches:
        peak = {
            'lat': float(match[0]),
            'lon': float(match[1]),
            'name': match[2],
            'height': int(match[3]),
            'id': int(match[4])
        }
        peaks.append(peak)
    return peaks

In [56]:
parse_peak_map(complete_file_content[0])

[{'lat': 46.92325547885,
  'lon': 9.9161259357656,
  'name': 'Schlappiner Spitz',
  'height': 2442,
  'id': 19661},
 {'lat': 46.925120861482,
  'lon': 9.907693862915,
  'name': 'Schlappiner Joch',
  'height': 2203,
  'id': 19660}]

In [57]:
peaks = []
for i in complete_file_content:
    peak = parse_peak_map(i)
    peaks.append(peak)

In [58]:
peaks

[[{'lat': 46.92325547885,
   'lon': 9.9161259357656,
   'name': 'Schlappiner Spitz',
   'height': 2442,
   'id': 19661},
  {'lat': 46.925120861482,
   'lon': 9.907693862915,
   'name': 'Schlappiner Joch',
   'height': 2203,
   'id': 19660}]]

In [63]:
## PArse Tourengänger
def parse_tour_partners(html_content: str):
    document = Selector(text=html_content)
    div = document.css('div.div15')
    partners = []
    ## Url is userID
    ## a contains the name
    for a in div.css('a'):
        url = a.css('::attr(href)').get()

        partners.append({
            'name': a.css('::text').get(),
            'url': url.split('/')[-2]
        })

    return partners

parse_tour_partners(complete_file_content[0])

[{'name': 'drixdrey', 'url': 'drixdrey'}]

In [64]:
partners = []
for i in complete_file_content:
    partner = parse_tour_partners(i)
    partners.append(partner)

In [65]:
partners

[[{'name': 'drixdrey', 'url': 'drixdrey'}]]

In [66]:
def parse_page_views(html_content: str):
    document = Selector(text=html_content)
    div = document.css('div[style="text-align:center;color:#666;font-size:0.814em"]')
    views = int(div.css('b::text').get())
    return views

parse_page_views(complete_file_content[0])

402